In [1]:
import json
import pandas as pd
import glob
import uuid
data_path = "../data/raw/2_stages"
interim_data_path = "../data/interim/2_stages"


In [2]:
files = glob.glob(f"{data_path}/*")
for i, file_name in enumerate(files):
    # print(file_name)
    data = json.load(open(file_name, "r"))[0]

    # reading data
    first_timestamp_miliseconds = data["time_AWARE"][0]
    last_timestamp_miliseconds = data["time_AWARE"][-1]
    time_diff_miliseconds =  last_timestamp_miliseconds - first_timestamp_miliseconds
    n_uncalibrated_samples = len(data["X_UnCal"])
    diff = time_diff_miliseconds / (n_uncalibrated_samples -1)
    data["time_uncalibrated"] = [first_timestamp_miliseconds + i * diff for i in range(n_uncalibrated_samples)]

    df_AWARE = pd.DataFrame({k: data[k] for k in ["X_AWARE", "Y_AWARE", "Z_AWARE", "time_AWARE"]})
    df_uncalibrated = pd.DataFrame({k: data[k] for k in ["X_UnCal", "Y_UnCal", "Z_UnCal", "X_Bias", "Y_Bias", "Z_Bias", "time_uncalibrated"]})

    df_aware_features = list(df_AWARE.columns)
    df_uncalibrated_features = list(df_uncalibrated.columns)
    device_id = data['DEVICE_ID_AWARE']
    stage = data["stage"]
    duration_seconds = time_diff_miliseconds / 1000
    aware_sample_rate = df_AWARE.size / duration_seconds
    uncalibrated_sample_rate = df_uncalibrated.size / duration_seconds
    user_id = file_name.split('_')[-1].split('.')[0]
    unique_id = uuid.uuid1()
    # print(f"{device_id = }")
    # print(f"{stage = }")
    # print(f"{duration_seconds =}s")
    # print(f"{aware_sample_rate = : .2f} Hz")
    # print(f"{df_aware_features = }")
    # print(f"{uncalibrated_sample_rate = :.2f} Hz")
    # print(f"{df_uncalibrated_features = }")
    # print(f"{user_id = }")
    df_AWARE.to_csv(f"{interim_data_path}/{device_id or 'no_id'}-stage_{stage}-{user_id}-aware.csv")
    df_uncalibrated.to_csv(f"{interim_data_path}/{device_id or 'no_id'}-stage_{stage}-{user_id}-uncalibrated.csv")
    print(f"{i}/{len(files)}")


0/2
1/2
